# 8. 분석

## 과제2. 기존 어린이 보호구역 중 교통안전시설물 우선 설치 지역 20개소를 제시하고 각 교통안전시설물의 위치 제시
### 8-1) 격자별 교통사고 위험지수 생성
> - **목표:** R 프로그래밍을 사용하여 최적 회귀 모형을 선정한 뒤 회귀계수를 이용하여 **격자별 교통사고 위험지수(hazard_index2)**를 설정한다. 
> - **y: 어린이 교통 사고 횟수, X: 교통 사고 요인** 
> - **변수 선택:** 선택된 모형에서 유의수준 0.1이하의 유의한 변수를 선택한다.
> - **모형 선택:** **y가 사고 횟수로 영과잉 자료**이므로 이전 과제에서 선택된 영과잉 음이항 모형을 사용한다.
> - **위험 지수 설정:** **최적 모
형의 회귀계수를 가중치로 사용**하여 격자별 교통사고 위험지수를 설정한다.

In [1]:
library(readxl)
library(MASS)
library(pscl)
library(xlsx)

Classes and Methods for R developed in the
Political Science Computational Laboratory
Department of Political Science
Stanford University
Simon Jackman
hurdle and zeroinfl functions by Achim Zeileis



In [69]:
all_data = read_excel('input/schoolzone_grid_divide.xlsx')
all_data<-as.data.frame(all_data)
head(all_data)

New names:
* `` -> ...1



,...1,geometry,coord_cent,geo_cent,gid,accident_02,schoolzone_cctv_cnt_09,bump_cnt_18,barrier_23,sign_cnt_16,⋯,crosswalk_cnt_17,trafficlight_cnt_19,bus_cnt_22,road_maxspeed_23,intersection_23,building_level_down_27,sports_cnt_29,pop_fluid_810_08,school_cnt_10,conjestion_freq_25
,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,"POLYGON ((127.016727045001 37.17607914622609, 127.0170100955357 37.17607914622609, 127.0170100955357 37.17630563946577, 127.016727045001 37.17630563946577, 127.016727045001 37.17607914622609))","[127.01686857026839, 37.17619239284593]",POINT (127.0168685702684 37.17619239284593),다사571087,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0.0000000,0,0
2,1,"POLYGON ((127.0170100955357 37.17607914622609, 127.0172931460704 37.17607914622609, 127.0172931460704 37.17630563946577, 127.0170100955357 37.17630563946577, 127.0170100955357 37.17607914622609))","[127.01715162080306, 37.17619239284593]",POINT (127.0171516208031 37.17619239284593),다사571087,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0.0000000,0,0
3,2,"POLYGON ((127.0172931460704 37.17607914622609, 127.0175761966052 37.17607914622609, 127.0175761966052 37.17630563946577, 127.0172931460704 37.17630563946577, 127.0172931460704 37.17607914622609))","[127.01743467133781, 37.17619239284593]",POINT (127.0174346713378 37.17619239284593),다사571087,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0.0000000,0,0
4,3,"POLYGON ((127.0175761966052 37.17607914622609, 127.0178592471399 37.17607914622609, 127.0178592471399 37.17630563946577, 127.0175761966052 37.17630563946577, 127.0175761966052 37.17607914622609))","[127.01771772187256, 37.17619239284593]",POINT (127.0177177218726 37.17619239284593),다사571087,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0.0000000,0,0
5,4,"POLYGON ((127.016727045001 37.17630563946577, 127.0170100955357 37.17630563946577, 127.0170100955357 37.17653213270545, 127.016727045001 37.17653213270545, 127.016727045001 37.17630563946577))","[127.01686857026839, 37.17641888608561]",POINT (127.0168685702684 37.17641888608561),다사571087,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0.0000000,0,0
6,5,"POLYGON ((127.0170100955357 37.17630563946577, 127.0172931460704 37.17630563946577, 127.0172931460704 37.17653213270545, 127.0170100955357 37.17653213270545, 127.0170100955357 37.17630563946577))","[127.01715162080306, 37.17641888608561]",POINT (127.0171516208031 37.17641888608561),다사571087,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0.4055556,0,0


In [63]:
data<-all_data[,-c(1:5)]
head(data,3)

,accident_02,schoolzone_cctv_cnt_09,bump_cnt_18,barrier_23,sign_cnt_16,pop_total_04,pop_child_05,ratio_child_0405,pop_working_06,crosswalk_cnt_17,trafficlight_cnt_19,bus_cnt_22,road_maxspeed_23,intersection_23,building_level_down_27,sports_cnt_29,pop_fluid_810_08,school_cnt_10,conjestion_freq_25
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,0,0,0,0,112,0,0,112,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,112,0,0,112,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,112,0,0,112,0,0,0,0,0,0,0,0,0,0


### 8-1-1) 영과잉 음이항 모형 (ZINB 모형)

In [33]:
zinb<-zeroinfl( accident_02 ~ pop_total_04 + 
    pop_child_05 + ratio_child_0405 + pop_working_06 + sign_cnt_16 + 
    crosswalk_cnt_17 + trafficlight_cnt_19 +schoolzone_cctv_cnt_09+
    sports_cnt_29 +   bus_cnt_22+ road_maxspeed_23  + pop_fluid_810_08 + school_cnt_10 + conjestion_freq_25 + intersection_23 + 
    barrier_23 |1, data=data, dist="negbin")
summary(zinb)


Call:
zeroinfl(formula = accident_02 ~ pop_total_04 + pop_child_05 + ratio_child_0405 + 
    pop_working_06 + sign_cnt_16 + crosswalk_cnt_17 + trafficlight_cnt_19 + 
    schoolzone_cctv_cnt_09 + sports_cnt_29 + bus_cnt_22 + road_maxspeed_23 + 
    pop_fluid_810_08 + school_cnt_10 + conjestion_freq_25 + intersection_23 + 
    barrier_23 | 1, data = data, dist = "negbin")

Pearson residuals:
    Min      1Q  Median      3Q     Max 
-0.6843 -0.3775 -0.3718 -0.3296  5.3930 

Count model coefficients (negbin with log link):
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)            -0.5058718  0.1529649  -3.307 0.000943 ***
pop_total_04           -0.0040157  0.0093895  -0.428 0.668884    
pop_child_05            0.0419028  0.0129841   3.227 0.001250 ** 
ratio_child_0405       -5.9050301  1.8746695  -3.150 0.001633 ** 
pop_working_06          0.0021624  0.0120319   0.180 0.857373    
sign_cnt_16             0.5329416  0.4051267   1.315 0.188344    
crosswalk_cn

In [64]:
data<- subset(data, select=-c(building_level_down_27)) #변수하나빼기

### 8-1-2) 위험 지수를 만들기 위한 변수 정규화
> - 각 변수를 최댓값으로 나누어 정규화 시켜준다.

In [67]:
#지수에 넣기 위해 변수 정규화
data_index2<-sapply(data, function(x) {x/max(x)})
summary(data_index2)

  accident_02      schoolzone_cctv_cnt_09  bump_cnt_18         barrier_23      
 Min.   :0.00000   Min.   :0.00000        Min.   :0.000000   Min.   :0.000000  
 1st Qu.:0.00000   1st Qu.:0.00000        1st Qu.:0.000000   1st Qu.:0.000000  
 Median :0.00000   Median :0.00000        Median :0.000000   Median :0.000000  
 Mean   :0.08046   Mean   :0.02538        Mean   :0.002155   Mean   :0.007184  
 3rd Qu.:0.00000   3rd Qu.:0.00000        3rd Qu.:0.000000   3rd Qu.:0.000000  
 Max.   :1.00000   Max.   :1.00000        Max.   :1.000000   Max.   :1.000000  
  sign_cnt_16       pop_total_04      pop_child_05     ratio_child_0405
 Min.   :0.00000   Min.   :0.00000   Min.   :0.00000   Min.   :0.0000  
 1st Qu.:0.00000   1st Qu.:0.00000   1st Qu.:0.00000   1st Qu.:0.0000  
 Median :0.00000   Median :0.04355   Median :0.00000   Median :0.0000  
 Mean   :0.01149   Mean   :0.09113   Mean   :0.08479   Mean   :0.1850  
 3rd Qu.:0.00000   3rd Qu.:0.14435   3rd Qu.:0.08025   3rd Qu.:0.3554  
 Max.   

### 8-1-3) 격자별 교통사고 위험지수 생성
> - 영과잉 음이항 모형을 통해 얻은 회귀 계수를 가중치로 사용하여 격자별 위험 지수 설정한다

In [74]:
# exponential 변환
data_index2$hazard_index2=exp(summary(zinb)$coefficients$count[1])+
                 exp(summary(zinb)$coefficients$count[3])*data_index2$pop_child_05+
                 exp(summary(zinb)$coefficients$count[4])*data_index2$ratio_child_0405+
                 exp(summary(zinb)$coefficients$count[7])*data_index2$crosswalk_cnt_17+
                 exp(summary(zinb)$coefficients$count[8])*data_index2$trafficlight_cnt_19+
                 exp(summary(zinb)$coefficients$count[10])*data_index2$bus_cnt_22

In [80]:
write.xlsx(data_index2,'input/data_index2.xlsx') # 위험지수를 포함한 데이터를 저장